In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import os
import sys
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [5]:
# list working dir

print(os.getcwd())

C:\Users\micha\OneDrive - České vysoké učení technické v Praze\Dokumenty\PycharmProjects\optimizin\General-Optimization-Solver\src\vrp\old


In [ ]:
# renaming results

import os
import json

def process_json_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                data = json.load(file)

            # Seřadíme záznamy ve slovníku
            data = {
                'benchmark_name': data.pop('benchmark_name'),
                'instance_name': os.path.splitext(filename)[0],  # Consistency with original source of Solomon instances
                'instance_kind': 'CVRPTW',
                'reference_solution': data.pop('reference_solution'),
                'data': data.pop('data'),
                'run_history': data.pop('run_history'),
            }
            
            # Uložíme změny zpět do JSON souboru
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
        # break

folder_path = "..\\..\\data\\VRPTW\\solomon_25\\"
process_json_files(folder_path)
